## Pre-processing eval data

MLQA Dataset, see excel file for explanation


In [ ]:
# process MLQA dataset to fit haystack doc stores, see explanation in the excel file

json_files = glob('/content/drive/MyDrive/Colab Notebooks/MLQA_V1/test/*.json')
#json_file = '/content/drive/MyDrive/Colab Notebooks/MLQA_V1/test/test-context-en-question-en.json'
#json_file = '/content/drive/MyDrive/Colab Notebooks/MLQA_V1/processed/MLQA_EN_EN.json'

all_files = []
for json_file in json_files:
  new_file = []
  with open(json_file) as f2:
    d2 = json.load(f2)
    
    for t in d2['data']:
      for i,c in enumerate(t['paragraphs']):
        tit = t['title']+'_'+str(i)
        doc_set = t['title']

        dic = {}
        dic['title'] = tit
        dic['doc_set'] = doc_set
        dic['num_docs'] = len(t['paragraphs'])
        dic['lang_ctx_qst'] = json_file[-19:-17] +'-'+ json_file[-7:-5]

        for q in c['qas']:
          dic['paragraphs'] = [{
                'context' : c['context'],
                'qas': [{
                        'answers': qx['answers'],
                        'id': qx['id'],
                        'question': qx['question'],
                        'is_impossible': False
                        } for qx in c['qas']]
                }]

        new_file.append(dic)
  all_files.extend(new_file)

In [ ]:
# 138 636 docs
# 158 083 qs

df = pd.DataFrame(all_files)

df.reset_index(drop=True, inplace=True)

groups = [df for _, df in df.groupby(['doc_set','lang_ctx_qst'])]
random.shuffle(groups)
df_shuffled = pd.concat(groups)

df_shuffled.reset_index(drop=True, inplace=True)


In [ ]:
len(df_shuffled)
#138 636 total rows


In [ ]:
slice1k = df_shuffled.head(70000)
fin_file = {}
fin_file['data'] = slice1k.to_dict(orient='records')


In [ ]:
fin_file = {}
fin_file['data'] = df_shuffled.to_dict(orient='records')


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/MLQA_V1/processed/MLQA_ALL_70k.json', 'w') as fp:
  json.dump(fin_file, fp)